In [1]:
# Imports
import pandas as pd
import transformers
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from scipy.special import softmax

: 

: 

In [2]:
# MODEL = f'cardiffnlp/twitter-roberta-base-sentiment-latest'
# tokenizer = AutoTokenizer.from_pretrained(MODEL)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
df = pd.read_json("280_spam_dropped.json")
df.head()

,comment,category,channel,video_url,video_id,channel_size,identity,score,results
0,8 months? Is that all.,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,0,1,1,0
1,impressed very effective,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,0,1,2,0
2,"No matter what, you have to use brushes and ag...",Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,0,1,1,0
3,"Good video BUT, why don't you ""articulate"" tha...",Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,0,1,1,0
4,"My wheels werenever done until 215k km, they l...",Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,0,1,2,0


0         1.0
1         2.0
2         1.0
3         1.0
4         2.0
         ... 
189757    0.0
189758    0.0
189759    1.0
189760    2.0
189761    2.0
Name: score, Length: 189762, dtype: float64

In [37]:


# # Load model and tokenizer
# model = RobertaForSequenceClassification.from_pretrained("roberta-base")
# tokenizer = RobertaTokenizer.from_pretrained("roberta-base")



# # Train/val split
# split_index = int(0.7 * len(df))
# train_df = df.iloc[:split_index]
# val_df = df.iloc[split_index:]

# # Get train and val data
# train_texts = train_df['comment'].tolist()
# train_labels = train_df['score'].tolist() 

# val_texts = val_df['comment'].tolist()
# val_labels = val_df['score'].tolist()

# # Encode data
# train_encodings = tokenizer(train_texts, truncation=True, padding=True)
# val_encodings = tokenizer(val_texts, truncation=True, padding=True) 

# # Create label ids per batch
# train_label_ids = [[label_map[label] for label in train_labels]]
# val_label_ids = [[label_map[label] for label in val_labels]]

# # Freeze base model
# for param in model.roberta.parameters():
#   param.requires_grad = False
  
# # Training loop
# optimizer = transformers.AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):

#   model.train()
#   for batch, labels in zip(train_encodings, train_label_ids):
  
#     outputs = model(batch, labels=labels)
#     loss = outputs[0]
#     loss.backward()
#     optimizer.step()
#     optimizer.zero_grad()
  
#   model.eval()
  
#   val_outputs = model(val_encodings, val_label_ids)
  
#   print(f"Epoch: {epoch+1}, Loss: {val_outputs.loss}, Accuracy: {val_outputs.accuracy}")

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: ''

In [45]:
# Load model and tokenizer
model = RobertaForSequenceClassification.from_pretrained("roberta-base") 
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Append texts and corresponding labels
# train_texts = []
# train_labels = [] 


# With 70/30 split
split_index = int(0.7 * len(df))

# train_df = df.iloc[:split_index]
# val_df = df.iloc[split_index:]

# Get train and validation data
train_df = df.iloc[:split_index]
val_df = df.iloc[split_index:]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
val_df.shape

(56929, 9)

In [49]:
labels_con = pd.to_numeric(df["score"])
df = df[df["score"]!=""]

In [57]:


train_texts = train_df['comment'].tolist()
train_labels = train_df['score'].tolist()

val_texts = val_df['comment'].tolist()
val_labels = val_df['score'].tolist()

# Encode data
train_encodings = tokenizer(train_texts, truncation=True, padding=True) 
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

batch_labels = []


# Freeze base model  
for param in model.roberta.parameters():
  param.requires_grad = False
  
# Training loop
optimizer = tf.optimizers.Adam(model.parameters(), lr=5e-5)


for epoch in range(3):

  model.train()
  for batch in train_encodings:  
    labels = val_df["score"].tolist()
    outputs = model(batch, labels=batch_labels)
    loss = outputs[0]
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
  model.eval()
  
  val_outputs = model(val_encodings, val_labels)

  print(f"Epoch: {epoch+1}, Loss: {val_outputs.loss}, Accuracy: {val_outputs.accuracy}")

KeyboardInterrupt: 

In [5]:
df.head()

,category,channel,video_url,video_id,comments,channel_size,identity,scores
0,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=Wqcy9do6YO0,Wqcy9do6YO0,[Wheel Cleaner is In production and pre-order ...,0,1,"[2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 0, 2, 0, 1, 1, ..."
1,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=NyNF0MfQN8s,NyNF0MfQN8s,[I know that on my 1122TST it also matters wha...,0,1,"[1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 2, 1, 0, 1, 1, ..."
2,Automobiles,https://www.youtube.com/@autofanaticcars,https://www.youtube.com/watch?v=x3vzurNYbyM,x3vzurNYbyM,"[Can you put it on top of PPF? Thanks, very ha...",0,1,"[1, 0, 1, 2, 0, 0, 1, 1, 1, 2, 2, 1, 2, 0, 1, ..."
3,Automobiles,https://www.youtube.com/@AutoAmateur,https://www.youtube.com/watch?v=qysq4YFOwPw,qysq4YFOwPw,[Hey australian Elon Musk can you show a littl...,0,1,"[1, 2, 1, 2, 2, 0, 2, 0, 1, 0, 0, 2, 1, 1, 2, ..."
4,Automobiles,https://www.youtube.com/@AutoAmateur,https://www.youtube.com/watch?v=B8zgCdINaAs,B8zgCdINaAs,[Excellent tutorial video. Congrats! Maybe two...,0,1,


In [6]:
len(df["comments"][0])

557

In [7]:
len(df["scores"][0])

557

In [8]:
df.to_json("this_took_280_minutes.json")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 317 entries, 0 to 316
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   category      317 non-null    object
 1   channel       317 non-null    object
 2   video_url     317 non-null    object
 3   video_id      317 non-null    object
 4   comments      317 non-null    object
 5   channel_size  317 non-null    int64 
 6   identity      317 non-null    int64 
 7   scores        317 non-null    object
dtypes: int64(2), object(6)
memory usage: 30.4+ KB
